In [5]:
import yfinance as yf 
from datetime import date, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os as os

In [6]:

today = date.today()

spark = SparkSession.builder \
        .appName('Company_Project_1.0.1') \
        .config("spark.jars", "mariadb-java-client-3.1.4.jar")\
        .getOrCreate()

23/10/20 15:26:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
DOCS = spark.read.format("jdbc")\
        .option("url","jdbc:mariadb://localhost:3306/lnd")\
        .option("driver", "org.mariadb.jdbc.Driver")\
        .option("dbtable", "DOCS")\
        .option("user", "ETL")\
        .option("password", 'Letmein2022!' )\
        .load()

In [18]:
MAX_DATE = DOCS.select(max(DOCS.Date)).alias('Max_Date').first()[0]

In [21]:
def GetStockData(stock, from_date):
    #gets stock data from today back 14yrs. stock and date inputs should be strings and progress should be booleen
    end_date = today.strftime("%Y-%m-%d")
    start_date = from_date
    progress = False
    stock_data = yf.download(stock, start_date, end_date, progress)
    stock_data["Date"] = stock_data.index
    stock_data = stock_data [["Date", "Open", "High", "Low", "Close",  "Volume"]]
    stock_data.reset_index(drop=True, inplace=True)
    stock_data = spark.createDataFrame(stock_data)
    stock_data = stock_data.withColumn("Name", lit(stock))
    return stock_data


In [22]:
DOCS_DATA = GetStockData('DOCS', MAX_DATE)

[*********************100%***********************]  1 of 1 completed


In [23]:
DOCS_DATA.show()

+-------------------+------------------+------------------+------------------+------------------+-------+----+
|               Date|              Open|              High|               Low|             Close| Volume|Name|
+-------------------+------------------+------------------+------------------+------------------+-------+----+
|2023-10-05 00:00:00|21.270000457763672|21.489999771118164|21.170000076293945|  21.3700008392334|1096400|DOCS|
|2023-10-06 00:00:00|21.200000762939453|22.075000762939453|21.200000762939453| 21.90999984741211|1676000|DOCS|
|2023-10-09 00:00:00| 21.59000015258789|22.110000610351562|              21.5|21.889999389648438|1262800|DOCS|
|2023-10-10 00:00:00|  21.8700008392334|22.700000762939453| 21.81999969482422| 22.40999984741211|1480300|DOCS|
|2023-10-11 00:00:00|22.329999923706055|22.639999389648438|22.150999069213867|22.270000457763672|1499200|DOCS|
|2023-10-12 00:00:00|22.280000686645508|22.280000686645508|21.420000076293945|21.670000076293945|1562100|DOCS|
|

In [25]:
DOCS_DATA.write \
    .format("jdbc") \
    .mode("append") \
    .option("driver", "org.mariadb.jdbc.Driver") \
    .option("url", "jdbc:mariadb://localhost:3306/lnd") \
    .option("dbtable", 'DOCS') \
    .option("user", "ETL") \
    .option("password", 'Letmein2022!') \
    .save()
